## DESCARGA DATOS DE LA API
### BALANCE ELECTRICO
En este notebook programo las funciones necesarias para hacer una descarga de datos de la api de REData, hacer las transformaciones necesarias para la limpieza de los dataframes y guardar esta información en un fichero csv.

In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.3`


import org.apache.spark.sql._
import org.apache.spark.sql.functions._


//Reducir numero logs
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)


//Circe para trabajar con JSON
import $ivy.`com.softwaremill.sttp.client::async-http-client-backend-monix:2.0.0-RC2`
import sttp.client.quick.{quickRequest, UriContext, backend}
import sttp.client.asynchttpclient.monix._


//Sttp para realizar request
import $ivy.`com.softwaremill.sttp.client::async-http-client-backend-monix:2.0.0-RC2`
import sttp.client.quick.{quickRequest, UriContext, backend}
import sttp.client.asynchttpclient.monix._
import sttp.client.basicRequest

//For adding extra dependenies
import $ivy.`org.typelevel::cats-core:1.6.0`

//Plotly
import $ivy.`org.plotly-scala::plotly-almond:0.7.1`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

import $ivy.$                                  



import org.apache.spark.sql._

import org.apache.spark.sql.functions._


//Reducir numero logs

import org.apache.log4j.{Level, Logger}

import $ivy.$                                                                        

import sttp.client.quick.{quickRequest, UriContext, backend}

import sttp.client.asynchttpclient.monix._


//Sttp para realizar request

import $ivy.$                                                                        

import sttp.client.quick.{quickRequest, UriContext, backend}

import sttp.client.asynchttpclient.monix._

import sttp.client.basicRequest

//For adding extra dependenies

import $ivy.$                               

//Plotly

import $ivy.$                                      

import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

In [2]:
//Iniciamos session de Spark
val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}

Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

spark: SparkSession = org.apache.spark.sql.SparkSession@5c453736

#### Construcción URI

La función createUri genera una URL a partir de los parámetros proporcionados, unos obligatorios y opcionales. Organiza estos en una cadena de consulta (queryString) y los añade a la URL base, que incluye la categoría y el widget deseado, para crear la URI completa con todos los parámetros de la solicitud.

In [3]:
import spark.implicits._ 

//Crear URL
def createUri(category: String, widget: String, start_date: String, end_date: String, time_trunc: String, lang: String, 
              geo_trunc: String = "", geo_limit: String = "", geo_ids: String = ""): String = {
    
    val baseUrl = s"https://apidatos.ree.es/$lang/datos/$category/$widget"
  
    // Crear un mapa con los parámetros obligatorios y opcionales
    val params = Map(
        "start_date" -> start_date,
        "end_date" -> end_date,
        "time_trunc" -> time_trunc,
        "geo_trunc" -> geo_trunc,
        "geo_limit" -> geo_limit,
        "geo_ids" -> geo_ids
      ).filter { case (_, v) => v.nonEmpty } // Filtrar los parámetros que no están vacíos

    //Especifico el orden concreto de los paramtros ya que, Scala map no te garantiza el orden
    val orderedKeys = Seq("start_date", "end_date", "time_trunc", "geo_trunc", "geo_limit", "geo_ids")

    // Unir los parámetros en una cadena
    val queryString = orderedKeys.flatMap(key => params.get(key).map(value => s"$key=$value")).mkString("&")
  
  s"$baseUrl?$queryString"
}

import spark.implicits._ 

//Crear URL

defined function createUri

In [4]:
val category = "balance"
val widget = "balance-electrico"
val time_trunc = "day"
val lang = "es"
val geo_trunc = "electric_system"
val geo_limit = "ccaa"
val geo_ids = "13"

val rango = (2011 to 2024).map( year => (year + "-01-01T00:00", year + "-12-31T23:59"))
val urisRango = rango.map {
    case (inicio, fin) => createUri(category, widget, inicio, fin, time_trunc,lang)
}

category: String = "balance"
widget: String = "balance-electrico"
time_trunc: String = "day"
lang: String = "es"
geo_trunc: String = "electric_system"
geo_limit: String = "ccaa"
geo_ids: String = "13"
rango: collection.immutable.IndexedSeq[(String, String)] = Vector(
  ("2011-01-01T00:00", "2011-12-31T23:59"),
  ("2012-01-01T00:00", "2012-12-31T23:59"),
  ("2013-01-01T00:00", "2013-12-31T23:59"),
  ("2014-01-01T00:00", "2014-12-31T23:59"),
  ("2015-01-01T00:00", "2015-12-31T23:59"),
  ("2016-01-01T00:00", "2016-12-31T23:59"),
  ("2017-01-01T00:00", "2017-12-31T23:59"),
  ("2018-01-01T00:00", "2018-12-31T23:59"),
  ("2019-01-01T00:00", "2019-12-31T23:59"),
  ("2020-01-01T00:00", "2020-12-31T23:59"),
  ("2021-01-01T00:00", "2021-12-31T23:59"),
  ("2022-01-01T00:00", "2022-12-31T23:59"),
  ("2023-01-01T00:00", "2023-12-31T23:59"),
  ("2024-01-01T00:00", "2024-12-31T23:59")
)
urisRango: collection.immutable.IndexedSeq[String] = Vector(
  "https://apidatos.ree.es/es/datos/balance/balance-el

### Descarga datos

La función **getDataApi** realiza una solicitud HTTP GET a la URL proporcionada (apiUrl) y devuelve la respuesta de la API. Utiliza la estructura Either para manejar posibles errores: si la solicitud es exitosa, retorna el contenido de la respuesta; en caso contrario, devuelve un mensaje de error.

La función **responseToDF** convierte la respuesta de la API, contenida en un Either, en un DataFrame de Spark. Si la respuesta es exitosa (Right), el contenido se transforma en un DataFrame JSON. Si ocurre un error (Left), se imprime el mensaje de error y se retorna un DataFrame vacío.

In [5]:
//Lanzar llamada y obtener guardar datos
def getDataApi(apiUrl: String): Either[String, String] = {
 
    val newRequest = basicRequest.get(uri"$apiUrl")
    val response = newRequest.send()
  
  response.body match {
    case Right(body) => 
      Right(body)
      
    case Left(error) => 
      Left(s"Error fetching data: $error")
  }
}

// Transformar response a DataFrame
def responseToDF(response: Either[String, String]): DataFrame = {
    response match {
    case Right(body) => 
        val rdd = spark.sparkContext.parallelize(Seq(body))
        spark.read.json(rdd)
    case Left(error) => 
        println(error)
        spark.emptyDataFrame
    }
}



defined function getDataApi
defined function responseToDF

In [6]:
val listBalanceJson = urisRango.map(getDataApi(_))

listBalnaceJson: collection.immutable.IndexedSeq[Either[String, String]] = Vector(
  Right(
    "{\"data\":{\"type\":\"Balance de energ\\u00eda el\\u00e9ctrica\",\"id\":\"bal1\",\"attributes\":{\"title\":\"Balance de energ\\u00eda el\\u00e9ctrica\",\"last-update\":\"2019-06-12T15:32:24.000+02:00\",\"description\":\"Balance el\\u00e9ctrico: asignaci\\u00f3n de unidades de producci\\u00f3n seg\\u00fan combustible principal. La producci\\u00f3n neta de las instalaciones no renovables e hidr\\u00e1ulicas UGH tiene descontados los consumos propios de las instalaciones. En dichos tipos de producci\\u00f3n una generaci\\u00f3n negativa indica que la electricidad horaria consumida para los usos de las plantas excede la producci\\u00f3n horaria de las instalaciones.\"},\"meta\":{\"cache-control\":{\"cache\":\"HIT\",\"expireAt\":\"2024-10-25T12:45:18\"}}},\"included\":[{\"type\":\"Renovable\",\"id\":\"Renovable\",\"attributes\":{\"title\":\"Renovable\",\"last-update\":\"2019-06-12T15:32:24.000+0

### Trafomración datos

La función **transformDataFrame** transforma el Dataframe optenido directamente de JSON en un formato más estructurado. 
Desanida los datos originales, extrayendo columnas relevantes como "Familia", "Tipo", "FechaCompleta", "Porcentaje" y "Valor". 
Luego, añade columnas adicionales para la fecha, hora y una etiqueta BajasEmisiones, que indica si el tipo de energía pertenece a un grupo de bajas emisiones previamente definido.

In [8]:
def transformDataFrame(df: DataFrame): DataFrame = {
    // Definir los tipos de energía de bajas emisiones
    val bajasEmisiones = Seq(
    "Solar fotovoltaica", 
    "Solar térmica", 
    "Nuclear", 
    "Hidroeólica", 
    "Eólica", 
    "Generación renovable", 
    "Otras renovables",
    "Hidráulica", 
    "Residuos renovables"
  )
    //Crear modelo desanidado
   df.withColumn("FamilyGroup", explode($"included"))
    .withColumn("TypeGroup", explode($"FamilyGroup.attributes.content"))
    .withColumn("Values", explode($"TypeGroup.attributes.values"))
    .select(
      $"FamilyGroup.type".as("Familia"),
      $"TypeGroup.type".as("Tipo"),
      $"Values.datetime".as("FechaCompleta"), 
      $"Values.percentage".as("Porcentaje"),
      $"Values.value".as("Valor")
    )
    .withColumn("FechaCompleta", $"FechaCompleta".cast("timestamp"))
    .withColumn("BajasEmisiones", $"Tipo".isin(bajasEmisiones: _*))
}

defined function transformDataFrame

In [7]:
val listModelsBalance = listBalnaceJson.map{
    json => transformDataFrame(responseToDF(json))
}
val modelBalance = listModelsBalance.reduce(_ union _)

json at cmd1.sc:52

16 / 16

json at cmd1.sc:52

16 / 16

json at cmd1.sc:52

...

json at cmd1.sc:52

16 / 16

json at cmd1.sc:52

16 / 16

listModelsBalance: collection.immutable.IndexedSeq[DataFrame] = Vector(
  [Familia: string, Tipo: string ... 6 more fields],
  [Familia: string, Tipo: string ... 6 more fields],
  [Familia: string, Tipo: string ... 6 more fields],
  [Familia: string, Tipo: string ... 6 more fields],
  [Familia: string, Tipo: string ... 6 more fields],
  [Familia: string, Tipo: string ... 6 more fields],
  [Familia: string, Tipo: string ... 6 more fields],
  [Familia: string, Tipo: string ... 6 more fields],
  [Familia: string, Tipo: string ... 6 more fields],
  [Familia: string, Tipo: string ... 6 more fields],
  [Familia: string, Tipo: string ... 6 more fields],
  [Familia: string, Tipo: string ... 6 more fields],
  [Familia: string, Tipo: string ... 6 more fields],
  [Familia: string, Tipo: string ... 6 more fields]
)
modelBalance: DataFrame = [Familia: string, Tipo: string ... 6 more fields]

In [8]:
modelBalance.coalesce(1).write
  .option("header", "true")
  .csv("data/modelosCsv/datasetTotalBalanceNacional11-24.csv")

: 

In [9]:
modelBalance.show()

show at cmd8.sc:1

1 / 1

show at cmd8.sc:1

4 / 4

show at cmd8.sc:1

20 / 20

+---------+----------+-------------------+-------------------+----------+----------+--------+--------------+
|  Familia|      Tipo|      FechaCompleta|         Porcentaje|     Valor|       Dia|    Hora|BajasEmisiones|
+---------+----------+-------------------+-------------------+----------+----------+--------+--------------+
|Renovable|Hidráulica|2010-12-31 23:00:00|  0.730243950800116|132922.642|2010-12-31|23:00:00|          true|
|Renovable|Hidráulica|2011-01-01 23:00:00|0.46094258667156873| 122475.16|2011-01-01|23:00:00|          true|
|Renovable|Hidráulica|2011-01-02 23:00:00| 0.5886483989680635|146861.638|2011-01-02|23:00:00|          true|
|Renovable|Hidráulica|2011-01-03 23:00:00| 0.5383243444180309|145876.053|2011-01-03|23:00:00|          true|
|Renovable|Hidráulica|2011-01-04 23:00:00| 0.4184631870402314|158946.134|2011-01-04|23:00:00|          true|
|Renovable|Hidráulica|2011-01-05 23:00:00|0.29923830274025337|113777.357|2011-01-05|23:00:00|          true|
|Renovable|Hidráuli

In [11]:
años.collect()

collect at cmd10.sc:1

224 / 224

collect at cmd10.sc:1

200 / 200

collect at cmd10.sc:1

200 / 200

collect at cmd10.sc:1

15 / 15

res10: Array[Row] = Array(
  [2010],
  [2011],
  [2012],
  [2013],
  [2014],
  [2015],
  [2016],
  [2017],
  [2018],
  [2019],
  [2020],
  [2021],
  [2022],
  [2023],
  [2024]
)

In [19]:
val prueba = listBalnaceJson.head match {
    case Right(body) => val primerjson = spark.sparkContext.parallelize(Seq(body))
        spark.read.json(primerjson)
    case Left(error) => 
        println(error)
        spark.emptyDataFrame
}

def responseToDF(response: Either[String, String]): DataFrame = {
    response match {
    case Right(body) => 
        val rdd = spark.sparkContext.parallelize(Seq(body))
        spark.read.json(rdd)
    case Left(error) => 
        println(error)
        spark.emptyDataFrame
    }
}

json at cmd18.sc:3

16 / 16

prueba: DataFrame = [data: struct<attributes: struct<description: string, last-update: string ... 1 more field>, id: string ... 2 more fields>, included: array<struct<attributes:struct<content:array<struct<attributes:struct<color:string,composite:boolean,description:string,last-update:string,magnitude:string,title:string,total:double,type:string,values:array<struct<datetime:string,percentage:double,value:double>>>,groupId:string,id:string,type:string>>,description:string,last-update:string,magnitude:string,title:string>,id:string,type:string>>]
defined function responseToDF

In [21]:
prueba.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- attributes: struct (nullable = true)
 |    |    |-- description: string (nullable = true)
 |    |    |-- last-update: string (nullable = true)
 |    |    |-- title: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- meta: struct (nullable = true)
 |    |    |-- cache-control: struct (nullable = true)
 |    |    |    |-- cache: string (nullable = true)
 |    |-- type: string (nullable = true)
 |-- included: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- attributes: struct (nullable = true)
 |    |    |    |-- content: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- attributes: struct (nullable = true)
 |    |    |    |    |    |    |-- color: string (nullable = true)
 |    |    |    |    |    |    |-- composite: boolean (nullable = true)
 |    |    |    |    |    |    |-- description: string (nullable = tru